This Notebook is the implementation of fine-tuning Falcon LLM on Financial Phrasebank Dataset for Sentiment Analysis task. Evaluation was performed on test dataset and results have been reported

### Install Required Libraries and Packages

In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7
!pip install huggingface_hub
!pip install -q datasets


import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from datasets import load_dataset
import pandas as pd
import random
from datasets import Dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 102.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00


### Login into Hugging Face Hub

In [2]:
from huggingface_hub import notebook_login
notebook_login()

### Load the Data 

In [4]:
### set display width as maximum for pandas dataframe to show the full data in the column
pd.set_option('display.max_colwidth', -1)

### load the dataset from Hugging Face hub, sentence_allagree version of the dataset will be used as mentioned in thesis document
dataset = load_dataset("financial_phrasebank",'sentences_allagree',split='train')


### Data is split into 75% Train and 25% Test
### The Target sentiment Label is picked in such a way (Stratified) that the proportion of sentiments is maintained in both Test and Train datasets

ds_split = dataset.train_test_split(test_size=0.25, stratify_by_column="label")

### Get the Train and Test Dataset from Hugging Face Dataset Dict object
ds_train = ds_split["train"]
ds_test = ds_split["test"]

train_dataset_dict = ds_train.train_test_split(test_size=0.1, stratify_by_column="label")
train_df = train_dataset_dict['train']
eval_df = train_dataset_dict['test']

### Convert Dataset into Pandas for Data preparation Steps
ds_train_pd = train_df.to_pandas()
ds_eval_pd = eval_df.to_pandas()
ds_test_pd = ds_test.to_pandas()


ds_train_pd['label'] = ds_train_pd['label'].map({0:'Negative',1:'Neutral',2:'Positive'})
ds_eval_pd['label'] = ds_eval_pd['label'].map({0:'Negative',1:'Neutral',2:'Positive'})


### List of Instructions
instructions = ['This statement has been extracted from a financial news article. Please provide your answer as either positive, negative or neutral.',
'The following financial news sentence has been provided as input. Determine if the sentiment is positive, negative, or neutral?',
'What is the sentiment of sentence extracted from financial news report? The provided options are positive, negative, or neutral.',
'Based on the text extracted from the financial news report, determine whether the sentiment is positive, negative, or neutral.',
'The sentiment of the text extracted from a financial news source needs to be evaluated. The sentiment could be positive, negative, or neutral.',
'This statement has been extracted from a financial news piece and its sentiment needs to be determined. Indicate whether it is positive, negative, or neutral.'
]
### create a list of 1698 instructions by randomly selecting instructions from above list and add it to train dataframe to create instructions column
ins_list = []
for i in range(len(ds_train_pd)):
  instr = random.choice(instructions)
  ins_list.append(instr)
ds_train_pd['instructions']  = ins_list


ins_list_eval = []
for i in range(len(ds_eval_pd)):
  instr_eval = random.choice(instructions)
  ins_list_eval.append(instr_eval)
ds_eval_pd['instructions']  = ins_list_eval

### Combine all elements to construct final prompt for fine-tuning
question_str = 'Determine the sentiment of text given as Input Text'
ds_train_pd['final_prompt'] = f'###Instruction:' + ds_train_pd['instructions'] + f'\n\n' + f'###Input Text:' + ds_train_pd['sentence'] + f'\n\n' + f'###Question:' + question_str + f'\n\n' + f'###Answer:' + ds_train_pd['label']


question_str = 'Determine the sentiment of text given as Input Text'
ds_eval_pd['final_prompt'] = f'###Instruction:' + ds_eval_pd['instructions'] + f'\n\n' + f'###Input Text:' + ds_eval_pd['sentence'] + f'\n\n' + f'###Question:' + question_str + f'\n\n' + f'###Answer:' + ds_eval_pd['label']

train_df = Dataset.from_pandas(ds_train_pd)
eval_df = Dataset.from_pandas(ds_eval_pd)



<ipython-input-4-00397069bbd1>:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


Generating train split:   0%|          | 0/2264 [00:00<?, ? examples/s]

### Fine-Tuning Framework Parameters¶

In [5]:
# The model that you want to train from the Hugging Face hub
model_name = "tiiuae/falcon-7b"



# Fine-tuned model name
new_model = "falcon-2-7b-fpb-rank-64-vj_v1.1"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64 ### RANK PARAMETER

# Alpha parameter for LoRA scaling
lora_alpha = 128

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = True
bf16 = False ### This parameter has been set to False since A100 GPU was not available during the execution of this code and hence V100 was used

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = 2000

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

device_map = {"": 0}

In [6]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# # Check GPU compatibility with bfloat16
# if compute_dtype == torch.float16 and use_4bit:
#     major, _ = torch.cuda.get_device_capability()
#     if major >= 8:
#         print("=" * 80)
#         print("Your GPU supports bfloat16: accelerate training with bf16=True")
#         print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/16.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

### Perform Fine-tuning

In [7]:
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load Falcon tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["query_key_value","dense","dense_h_to_4h","dense_4h_to_h"]
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
    evaluation_strategy="epoch"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_df,
    eval_dataset=eval_df,
    peft_config=peft_config,
    dataset_text_field="final_prompt",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/1528 [00:00<?, ? examples/s]

Map:   0%|          | 0/170 [00:00<?, ? examples/s]

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.852900,0.830095


In [8]:
# The model that you want to train from the Hugging Face hub
model_name = "tiiuae/falcon-7b"



# Fine-tuned model name
new_model = "falcon-2-7b-fpb-rank-64-vj_v1.1"



# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Save the fine-tuned model and push it to hugging Face hub

In [9]:
!huggingface-cli login

model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Your token has been saved in your con

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.89G [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/vijayshankar245/falcon-2-7b-fpb-rank-64-vj_v1.1/commit/ebf184d23179b8b7791e51ba3d7a0b7371fa2a54', commit_message='Upload tokenizer', commit_description='', oid='ebf184d23179b8b7791e51ba3d7a0b7371fa2a54', pr_url=None, pr_revision=None, pr_num=None)

### Evaluation

### construct the prompt for test dataset

In [10]:
question_str = 'Determine the sentiment of text given as Input Text'
ds_test_pd['final_prompt'] = f'###Input Text:' + ds_test_pd['sentence'] + f'\n\n' + f'###Question:' + question_str
test_df = Dataset.from_pandas(ds_test_pd)

### Load the fine-tuned model for evaluation

In [13]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)
model = "vijayshankar245/falcon-2-7b-fpb-rank-64-vj_v1.1"
tokenizer = AutoTokenizer.from_pretrained(model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
# Run text generation pipeline with our next model

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer,return_full_text=False,temperature=0.6,top_p =0.9,max_new_tokens = 10)

tokenizer_config.json:   0%|          | 0.00/248 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/16.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.89G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

### Evaluate the LLM

In [14]:
import time
start_time = time.time()
from transformers.pipelines.pt_utils import KeyDataset
output_list = []
j=0
for i in pipe(KeyDataset(test_df, "final_prompt")):
  j=j+1
  print(j,':',i[0]['generated_text'])
  output_list.append(i[0]['generated_text'])
  df = pd.DataFrame({'output': output_list})
print("--- %s seconds ---" % (time.time() - start_time))

1 : 

###Answer:Negative

###Input Text:
2 : 

###Answer:Positive

###Input Text:
3 : 

###Answer:Positive

###Input Text:
4 : 

###Answer:Positive

###Input Text:
5 : 

###Answer:Negative

###Input Text:
6 : 

###Answer:Neutral

###Input Text
7 : 

###Answer:Negative

###Input Text:
8 : 

###Answer:Negative

###Input Text:
9 : 

###Answer:Negative

###Input Text:
10 : 

###Answer:Positive

###Input Text:
11 : 

###Answer:Neutral

###Input Text
12 : 

###Answer:Neutral

###Input Text
13 : 

###Answer:Negative

###Input Text:
14 : 

###Answer:Neutral

###Input Text
15 : 

###Answer:Positive

###Input Text:
16 : 

###Answer:Neutral

###Input Text
17 : 

###Answer:Positive

###Input Text:
18 : 

###Answer:Neutral

###Input Text
19 : 

###Answer:Positive

###Input Text:
20 : 

###Answer:Neutral

###Input Text
21 : 

###Answer:Neutral

###Input Text
22 : 

###Answer:Positive

###Input Text:
23 : 

###Answer:Positive

###Input Text:
24 : 

###Answer:Positive

###Input Text:
25 : 

###Answer:

In [ ]:
len(output_list)

566

### Perform Post Processing

In [ ]:
op_list = []
for txt in output_list:
  int1 = txt.replace(f"\n\n###Answer:", "")
  int2 = int1.replace(f"\n","")
  op_list.append(int2)

Add the predicted output to test dataset and perform evaluation

In [13]:
ds_test_pd['pred_sentiment'] = op_list
ds_test_pd['pred_sentiment'] = ds_test_pd['pred_sentiment'].map({'Negative':0,'Neutral':1,'Positive':2})
from sklearn.metrics import classification_report
print(classification_report(ds_test_pd['label'], ds_test_pd['pred_sentiment']))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97        76
           1       0.99      0.97      0.98       348
           2       0.95      0.97      0.96       142

    accuracy                           0.97       566
   macro avg       0.96      0.98      0.97       566
weighted avg       0.97      0.97      0.97       566



### Confusion Matrix

In [12]:
from sklearn.metrics import multilabel_confusion_matrix
tst = multilabel_confusion_matrix(ds_test_pd['label'], ds_test_pd['pred_sentiment'],labels=[0,1,2])

In [14]:
tst

array([[[486,   4],
        [  0,  76]],

       [[214,   4],
        [ 12, 336]],

       [[416,   8],
        [  4, 138]]])